# Load-Dependent Multi-Class PS Queues

This example demonstrates load-dependent queues with multiple job classes and PS (Processor Sharing) scheduling.

## Features:
- Two closed classes: Class1 (4 jobs), Class2 (2 jobs)
- Three stations: Delay + two PS queues with load dependence
- Each queue has 3 servers with load dependence: min(total_jobs, servers)
- Compares load-independent model vs load-dependent model
- Demonstrates different solver methods: CTMC, NC, MVA

## Load Dependence:
- **Load-Independent**: Fixed number of servers regardless of population
- **Load-Dependent**: Active servers = min(population, 3)
- Creates realistic modeling of servers that scale with demand

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.ctmc.*
import jline.util.matrix.*

In [ ]:
// Model parameters
val N = 4  // number of jobs in Class1
val c = 3  // number of servers per queue

// Create load-independent model for comparison
val model = Network("model")
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PS)
val node3 = Queue(model, "Queue2", SchedStrategy.PS)

// Create job classes
val jobclass1 = ClosedClass(model, "Class1", N, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", N / 2, node1, 0)  // N/2 = 2 jobs

// Set service times
node1.setService(jobclass1, Exp.fitMean(1.0))  // mean = 1.0
node1.setService(jobclass2, Exp.fitMean(2.0))  // mean = 2.0

node2.setService(jobclass1, Exp.fitMean(1.5))  // mean = 1.5
node2.setService(jobclass2, Exp.fitMean(2.5))  // mean = 2.5
(node2 as Queue).numberOfServers = c

node3.setService(jobclass1, Exp.fitMean(3.5))  // mean = 3.5
node3.setService(jobclass2, Exp.fitMean(4.5))  // mean = 4.5
(node3 as Queue).numberOfServers = c

// Create routing matrix (serial routing for each class)
val P = model.initRoutingMatrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)
model.link(P)

In [ ]:
// Solve load-independent model with MVA
println("=== Load-Independent Model (MVA) ===")
val solverMva = MVA(model, "method", "exact")
val msT = solverMva.avgTable
msT.print()

In [ ]:
// Create load-dependent model
val ldmodel = Network("ldmodel")
val ldnode1 = Delay(ldmodel, "Delay")
val ldnode2 = Queue(ldmodel, "Queue1", SchedStrategy.PS)
val ldnode3 = Queue(ldmodel, "Queue2", SchedStrategy.PS)

// Create job classes
val ldjobclass1 = ClosedClass(ldmodel, "Class1", N, ldnode1, 0)
val ldjobclass2 = ClosedClass(ldmodel, "Class2", N / 2, ldnode1, 0)

// Set service times (same as load-independent model)
ldnode1.setService(ldjobclass1, Exp.fitMean(1.0))
ldnode1.setService(ldjobclass2, Exp.fitMean(2.0))

ldnode2.setService(ldjobclass1, Exp.fitMean(1.5))
ldnode2.setService(ldjobclass2, Exp.fitMean(2.5))

ldnode3.setService(ldjobclass1, Exp.fitMean(3.5))
ldnode3.setService(ldjobclass2, Exp.fitMean(4.5))

// Set load dependence: min(total_jobs, c) servers
val totalJobs = N + N / 2  // 4 + 2 = 6 total jobs
val alpha = DoubleArray(totalJobs + 1) { i ->
    kotlin.math.min(kotlin.math.max(i, 1).toDouble(), c.toDouble())
}
ldnode2.setLoadDependence(alpha.toList())
ldnode3.setLoadDependence(alpha.toList())

println("Load dependence function: ${alpha.toList()}")

// Create routing matrix
val ldP = ldmodel.initRoutingMatrix()
ldP.set(ldjobclass1, ldjobclass1, ldnode1, ldnode2, 1.0)
ldP.set(ldjobclass1, ldjobclass1, ldnode2, ldnode3, 1.0)
ldP.set(ldjobclass1, ldjobclass1, ldnode3, ldnode1, 1.0)
ldP.set(ldjobclass2, ldjobclass2, ldnode1, ldnode2, 1.0)
ldP.set(ldjobclass2, ldjobclass2, ldnode2, ldnode3, 1.0)
ldP.set(ldjobclass2, ldjobclass2, ldnode3, ldnode1, 1.0)
ldmodel.link(ldP)

In [ ]:
// Solve with CTMC (exact solution)
println("\n=== Load-Dependent Model (CTMC) ===")
try {
    val solverCtmc = CTMC(ldmodel)
    val lldAvgTableCTMC = solverCtmc.avgTable
    lldAvgTableCTMC.print()
} catch (e: Exception) {
    println("CTMC solver failed: ${e.message}")
}

In [ ]:
// Try NC solver with different methods
println("\n=== NC Solver Results ===")

// Skip 'exact' method as it's not supported for load-dependent models
val ncMethods = listOf("rd", "nrp", "nrl")
for (method in ncMethods) {
    try {
        val solver = NC(ldmodel, "method", method)
        val result = solver.avgTable
        println("\nNC $method:")
        result.print()
    } catch (e: Exception) {
        println("NC $method failed: ${e.message}")
    }
}

In [ ]:
// Try MVA solvers for load-dependent model
println("\n=== MVA Solver Results ===")
val mvaMethods = listOf("exact", "qd")
for (method in mvaMethods) {
    try {
        val solver = MVA(ldmodel, "method", method)
        val result = solver.avgTable
        println("\nMVA $method:")
        result.print()
    } catch (e: Exception) {
        println("MVA $method failed: ${e.message}")
    }
}

## Results Analysis

This example demonstrates the effect of load dependence on queue performance:

1. **Load-Independent Model**: Each queue has a fixed number of servers (3) regardless of population
2. **Load-Dependent Model**: Number of active servers depends on total population: min(population, 3)

The load dependence affects system performance by:
- Limiting the effective service capacity when population is low
- Providing full service capacity only when enough jobs are present
- This creates more realistic modeling of servers that scale with demand

Different solvers (CTMC, NC, MVA) provide various numerical methods for solving the same model, allowing comparison of accuracy and computational efficiency.